# Neo4j GraphDatabase API for Connection and Cypher usage

In [1]:
from neo4j.v1 import GraphDatabase, basic_auth
import pandas as pd

In [2]:
driver = GraphDatabase.driver(
    "bolt://54.167.174.205:33793", 
    auth=basic_auth("neo4j", "swords-hilltops-instructors"))
session = driver.session()

cypher_query = '''
MATCH (u:User {name: "Andrew Freeman"})-[r:RATED]->(m:Movie)
WITH u, avg(r.rating) AS mean

MATCH (u)-[r:RATED]->(m:Movie)-[:IN_GENRE]->(g:Genre)
WHERE r.rating > mean

WITH u, g, COUNT(*) AS score

MATCH (g)<-[:IN_GENRE]-(rec:Movie)
WHERE NOT EXISTS((u)-[:RATED]->(rec))

RETURN rec.title AS recommendation, rec.year AS year, COLLECT(DISTINCT g.name) AS genres, SUM(score) AS sscore
ORDER BY sscore DESC LIMIT 10
'''

results = session.run(cypher_query)

In [3]:
df = pd.DataFrame([r.values() for r in results], columns=results.keys())
df.fillna(0,inplace=True)
df['year'] = df['year'].astype(int)

In [4]:
df

,recommendation,year,genres,sscore
0,Mars Needs Moms,2011,"[Children, Animation, Sci-Fi, Action, Comedy, ...",155
1,"Wonderful World of the Brothers Grimm, The",1962,"[Romance, Musical, Children, Fantasy, Animatio...",155
2,War of the Worlds,2005,"[Sci-Fi, Action, Thriller, Adventure]",155
3,Aqua Teen Hunger Force Colon Movie Film for Th...,2007,"[Mystery, Fantasy, Animation, Sci-Fi, Action, ...",151
4,Futurama: Bender's Game,2008,"[Fantasy, Animation, Sci-Fi, Action, Comedy, A...",149
5,Kingsglaive: Final Fantasy XV,0,"[Fantasy, Animation, Sci-Fi, Action, Adventure...",149
6,Meet the Robinsons,2007,"[Children, Animation, Sci-Fi, Action, Comedy, ...",144
7,Chicken Little,2005,"[Children, Animation, Sci-Fi, Action, Comedy, ...",144
8,Robots,2005,"[Children, Fantasy, Animation, Sci-Fi, Comedy,...",142
9,Super Mario Bros.,1993,"[Children, Fantasy, Sci-Fi, Action, Comedy, Ad...",142
